In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

aug_data=ImageDataGenerator(rotation_range=20,horizontal_flip=True,width_shift_range=0.1,shear_range = 0.2,height_shift_range=0.1,zoom_range=0.2,brightness_range = (0.5, 1.5))
aug_data.fit(x_train)

169009152/169001437 [==============================] - 4s 0us/step


In [2]:

model = Sequential()
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Flattening the pooled image pixels
model.add(Flatten())

model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dropout(0.5))
model.add(Dense(units= 100, activation='softmax'))
'''
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
gvs = optimizer.compute_gradients(cost)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs)
'''
sgd=SGD(learning_rate=0.01,momentum=0.9,clipnorm=1)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint = ModelCheckpoint("VGG_DropOut_SGD_weights_amit.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto',restore_best_weights=True)


In [3]:
model.fit(aug_data.flow(x_train, y_train, batch_size=128), batch_size=128, epochs=100, verbose=1, validation_data=(x_test, y_test),callbacks=[checkpoint,early])


Epoch 1/100
391/391 [==============================] - ETA: 0s - loss: 4.7611 - accuracy: 0.0154
Epoch 00001: val_accuracy improved from -inf to 0.00990, saving model to VGG_DropOut_SGD_weights_amit.hdf5
391/391 [==============================] - 152s 388ms/step - loss: 4.7611 - accuracy: 0.0154 - val_loss: 4.6163 - val_accuracy: 0.0099
Epoch 2/100
391/391 [==============================] - ETA: 0s - loss: 4.4492 - accuracy: 0.0249
Epoch 00002: val_accuracy improved from 0.00990 to 0.01510, saving model to VGG_DropOut_SGD_weights_amit.hdf5
391/391 [==============================] - 150s 382ms/step - loss: 4.4492 - accuracy: 0.0249 - val_loss: 4.5542 - val_accuracy: 0.0151
Epoch 3/100
391/391 [==============================] - ETA: 0s - loss: 4.2988 - accuracy: 0.0438
Epoch 00003: val_accuracy improved from 0.01510 to 0.04110, saving model to VGG_DropOut_SGD_weights_amit.hdf5
391/391 [==============================] - 149s 382ms/step - loss: 4.2988 - accuracy: 0.0438 - val_loss: 4.3492 

In [5]:
y_pred=model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [6]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

              precision    recall  f1-score   support

           0       0.82      0.72      0.77       100
           1       0.77      0.55      0.64       100
           2       0.50      0.34      0.40       100
           3       0.42      0.15      0.22       100
           4       0.28      0.16      0.20       100
           5       0.63      0.31      0.42       100
           6       0.60      0.52      0.56       100
           7       0.46      0.60      0.52       100
           8       0.75      0.57      0.65       100
           9       0.65      0.59      0.62       100
          10       0.50      0.26      0.34       100
          11       0.54      0.29      0.38       100
          12       0.45      0.60      0.52       100
          13       0.50      0.36      0.42       100
          14       0.37      0.37      0.37       100
          15       0.47      0.47      0.47       100
          16       0.58      0.54      0.56       100
          17       0.62    

In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)


model = Sequential()
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Flattening the pooled image pixels
model.add(Flatten())

model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dropout(0.5))
model.add(Dense(units= 100, activation='softmax'))

model.load_weights("VGG_DropOut_SGD_weights_amit.hdf5")

y_pred=model.predict_classes(x_test)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

Using TensorFlow backend.




              precision    recall  f1-score   support

           0       0.82      0.72      0.77       100
           1       0.77      0.55      0.64       100
           2       0.50      0.34      0.40       100
           3       0.42      0.15      0.22       100
           4       0.28      0.16      0.20       100
           5       0.63      0.31      0.42       100
           6       0.60      0.52      0.56       100
           7       0.46      0.60      0.52       100
           8       0.75      0.57      0.65       100
           9       0.65      0.59      0.62       100
          10       0.50      0.26      0.34       100
          11       0.54      0.29      0.38       100
          12       0.45      0.60      0.52       100
          13       0.50      0.36      0.42       100
          14       0.37      0.37      0.37       100
          15       0.47      0.47      0.47       100
          16       0.58      0.54      0.56       100
          17       0.62  

In [2]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)


model = Sequential()
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))
model.add(Dropout(0.5))

# Flattening the pooled image pixels
model.add(Flatten())

model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))

model.add(Dropout(0.5))
model.add(Dense(units= 100, activation='softmax'))

model.load_weights("../weights/VGG_DropOut_SGD_weights_amit.hdf5")

y_true = y_test.argmax(-1)
y_pred = model.predict(x_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))


              precision    recall  f1-score   support

           0       0.82      0.72      0.77       100
           1       0.77      0.55      0.64       100
           2       0.50      0.34      0.40       100
           3       0.42      0.15      0.22       100
           4       0.28      0.16      0.20       100
           5       0.63      0.31      0.42       100
           6       0.60      0.52      0.56       100
           7       0.46      0.60      0.52       100
           8       0.75      0.57      0.65       100
           9       0.65      0.59      0.62       100
          10       0.50      0.26      0.34       100
          11       0.54      0.29      0.38       100
          12       0.45      0.60      0.52       100
          13       0.50      0.36      0.42       100
          14       0.37      0.37      0.37       100
          15       0.47      0.47      0.47       100
          16       0.58      0.54      0.56       100
          17       0.62   